In [1]:
from infinipy.stateblock import StateBlock
import uuid
from copy import deepcopy

from infinipy.statement import Statement,CompositeStatement

from typing import List, Optional, Dict, Any, Union, Tuple
from infinipy.options import Option
from infinipy.worldstatement import WorldStatement
from dataclasses import dataclass, fields
from infinipy.actions import Action

In [2]:
from infinipy.demos.goap.lockkeydemo import create_demo

In [6]:
entities,statements,actions, (starting_state,goal_state) = create_demo()

In [7]:
from infinipy.goap import GOAP

goap = GOAP (actions)
goap.backward_solve(starting_state,goal_state)

Backward solution found: character_pick_key <- character_unlock_door <- character_drop_key <- character_pick_key <- character_lock_door <- character_unlock_door <- character_open_door
3
Backward solution found: character_pick_key <- character_unlock_door <- character_open_door
7
Backward solution found: character_pick_key <- character_unlock_door <- character_drop_key <- character_pick_key <- character_open_door
5
Backward solution found: character_pick_key <- character_unlock_door <- character_drop_key <- character_open_door
6
Backward solution found: character_pick_key <- character_unlock_door <- character_open_door <- character_lock_door
6
Backward solution found: character_pick_key <- character_unlock_door <- character_open_door <- character_drop_key <- character_close_door <- character_pick_key <- character_open_door <- character_lock_door
2
Backward solution found: character_pick_key <- character_unlock_door <- character_drop_key <- character_pick_key <- character_open_door <- ch

In [ ]:
goap = GOAP (actions)

goap.forward_solve(starting_state,goal_state)

Trying action: character_pick_key
Trying action: character_unlock_door
Trying action: character_open_door
Solution found: character_pick_key -> character_unlock_door -> character_open_door


In [ ]:
end_world = goap.terminal_world
goap.forward_solve(end_world,starting_state)

Trying action: character_close_door
Trying action: character_open_door
Trying action: character_close_door
Trying action: character_lock_door
Trying action: character_unlock_door
Trying action: character_drop_key
Trying action: character_pick_key
Trying action: character_drop_key
Trying action: character_close_door
Trying action: character_open_door
Trying action: character_pick_key
Trying action: character_pick_key
Trying action: character_lock_door
Trying action: character_unlock_door
Trying action: character_drop_key
Solution found: character_close_door -> character_lock_door -> character_drop_key


In [ ]:
goap.backward_solve(end_world,starting_state)

Backward solution found: character_close_door <- character_lock_door <- character_drop_key
7
Backward solution found: character_drop_key <- character_close_door <- character_pick_key <- character_lock_door <- character_drop_key
5
